<a href="https://colab.research.google.com/github/zinebzannouti/Web-Scraping/blob/main/Exercice/3_Web_Scraping_avec_Scrapy_NO_IMAGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Web Scraping using Scrapy**

**Installer Scrapy :**

In [ ]:
# install scrapy
!pip install Scrapy

**Créer un projet**

- Avant de commencer à gscraper, vous devrez configurer un nouveau projet Scrapy.

- Entrez un répertoire dans lequel vous souhaitez stocker votre code et exécutez :

In [ ]:
# create files for learning
!scrapy startproject firstproject

New Scrapy project 'firstproject', using template directory '/usr/local/lib/python3.8/dist-packages/scrapy/templates/project', created in:
    /content/firstproject

You can start your first spider with:
    cd firstproject
    scrapy genspider example example.com


- Cela créera un répertoire 'firstproject' avec le contenu suivant :

In [ ]:
# get the current working directory
import os
os.getcwd()

'/content'

In [ ]:
# change working directories
os.chdir('/content/firstproject/firstproject/spiders')
os.getcwd()

'/content/firstproject/firstproject/spiders'

# **1 - Extraire les quotes :**

**Notre premier Spider :**

- Les spiders sont des classes que vous définissez et que Scrapy utilise pour extraire des informations d'un site Web (ou d'un groupe de sites Web).
- Ils doivent sous-classer Spideret définir les requêtes initiales à effectuer, éventuellement comment suivre les liens dans les pages et comment analyser le contenu de la page téléchargée pour extraire des données.

In [ ]:
# create quotes_spider.py and save it under the firstproject/firstproject/spiders directory
%%writefile -a quotes_spider.py
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'https://quotes.toscrape.com/page/1/',
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
        
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.css('small.author::text').get(),
                'tags': quote.css('div.tags a.tag::text').getall(),
            }

        next_page = response.css('li.next a::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

Writing quotes_spider.py


Comme vous pouvez le voir, notre Spider sous-classe **scrapy.Spider** et définit certains attributs et méthodes :

- **name**: identifie le spider. Il doit être unique au sein d'un projet, c'est-à-dire que vous ne pouvez pas définir le même nom pour différentes spiders.


- **response.css()** : Le résultat de l'exécution response.css('div.quote')est un objet de type liste appelé SelectorList, qui représente une liste d' Selectorobjets qui entourent les éléments XML/HTML et vous permettent d'exécuter d'autres requêtes pour affiner la sélection ou extraire les données

- **yield** : Un Spider Scrapy génère généralement de nombreux dictionnaires contenant les données extraites de la page. Pour ce faire, nous utilisons le yield pour spécifier ce qu'on veut extraire'



-  la **parse()** méthode recherche le lien vers la page suivante, construit une URL absolue complète à l'aide de la urljoin()méthode (puisque les liens peuvent être relatifs) et génère une nouvelle requête vers la page suivante, s'enregistrant comme rappel à gérer l'extraction des données pour la page suivante et de continuer à parcourir toutes les pages.

Comment faire fonctionner notre Spider :

- Pour faire fonctionner notre araignée, accédez au répertoire de niveau supérieur du projet et exécutez :

- Cette commande exécute l'araignée avec le nom quotesque nous venons d'ajouter, qui enverra des requêtes pour le quotes.toscrape.com domaine.

In [ ]:
!scrapy crawl quotes

**Stockage des données scrapées**

- Le moyen le plus simple de stocker les données extraites consiste à utiliser Feed exports , avec la commande suivante : !scrapy crawl quotes -O quotes.json

- Cela générera un quotes.json fichier contenant tous les éléments récupérés, sérialisés en JSON .

In [ ]:
!scrapy crawl quotes -O quotes.json

**Convert our data to CSV format :**

In [ ]:
import pandas as pd
df = pd.read_json('/content/firstproject/firstproject/spiders/quotes.json')
df.to_csv('data.csv')
df.head(10)

,text,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"
5,“Try not to become a man of success. Rather be...,Albert Einstein,"[adulthood, success, value]"
6,“It is better to be hated for what you are tha...,André Gide,"[life, love]"
7,"“I have not failed. I've just found 10,000 way...",Thomas A. Edison,"[edison, failure, inspirational, paraphrased]"
8,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt,[misattributed-eleanor-roosevelt]
9,"“A day without sunshine is like, you know, nig...",Steve Martin,"[humor, obvious, simile]"


# **2 - Extraire des données sur les Auteurs :**

**On va suivre faire la même procedure pour extraire les données des auteurs :**

## **TO DO**
- Essayer de suivre la meme procedure qu'on a déja fait pour extraire les quotes , mais cette fois pour extraire des données sur les auteurs (nom , date de naissance , bio)
- Remplissez les champs fill_here dans le script author_spider.py

In [ ]:
%%writefile -a author_spider.py
import scrapy


class AuthorSpider(scrapy.Spider):
    name = 'author'

    start_urls = ['https://quotes.toscrape.com/']

    def parse(self, response):
        author_page_links = response.css('.author + a')
        yield from response.follow_all(author_page_links, self.parse_author)

        pagination_links = response.css('li.next a')
        yield from response.follow_all(pagination_links, self.parse)

    def parse_author(self, response):
        def extract_with_css(query):
            return response.css(query).get(default='').strip()

        yield {
            'name': #fill_here,
            'birthdate': #fill_here,
            'bio': #fill_here,
        }

Writing author_spider.py


In [ ]:
!scrapy crawl author

In [ ]:
!scrapy crawl author -O authors.json

In [ ]:
df = pd.read_json('/content/firstproject/firstproject/spiders/authors.json')
df.to_csv('data.csv')
df.head(10)

,name,birthdate,bio
0,Albert Einstein,"March 14, 1879","In 1879, Albert Einstein was born in Ulm, Germ..."
1,André Gide,"November 22, 1869",André Paul Guillaume Gide was a French author ...
2,Marilyn Monroe,"June 01, 1926",Marilyn Monroe (born Norma Jeane Mortenson; Ju...
3,Eleanor Roosevelt,"October 11, 1884",Anna Eleanor Roosevelt was an American politic...
4,J.K. Rowling,"July 31, 1965",See also: Robert GalbraithAlthough she writes ...
5,Steve Martin,"August 14, 1945","Stephen Glenn ""Steve"" Martin is an American ac..."
6,Jane Austen,"December 16, 1775",Jane Austen was an English novelist whose work...
7,Thomas A. Edison,"February 11, 1847","Thomas Alva Edison was an American inventor, s..."
8,Allen Saunders,"April 24, 1899","Allen Saunders was an American writer, journal..."
9,Mark Twain,"November 30, 1835","Samuel Langhorne Clemens, better known by his ..."


**- Voici le lien Colab :** https://colab.research.google.com/github/zinebzannouti/Web-Scraping/blob/main/Exercice/3_Web_Scraping_avec_Scrapy.ipynb#scrollTo=cgfkCmYjt-HD